## Day 03: Data Cleaning

Objective (Day 3):

Standardize dates, clean missing values, and prepare the dataset
for time-series analysis and dashboarding.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
climate_df = pd.read_csv("mumbai_climate.csv")
climate_df

,Date,Rain,Temp Max,Temp Min
0,01-01-1951,NaN,28.53000069,14.53999996
1,02-01-1951,NaN,28.85000038,14.47999954
2,03-01-1951,NaN,30.65999985,14.43000031
3,04-01-1951,NaN,30.13999939,14.35999966
4,05-01-1951,NaN,29.18000031,13.34000015
...,...,...,...,...
26801,18-06-2024,Tr,34.3,26.6
26802,19-06-2024,20,34.8,25.5
26803,45463,18,33.1,25.4
26804,45464,4,30.9,26.7


In [3]:
climate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26806 entries, 0 to 26805
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      26806 non-null  object
 1   Rain      1238 non-null   object
 2   Temp Max  26806 non-null  object
 3   Temp Min  26805 non-null  object
dtypes: object(4)
memory usage: 837.8+ KB


In [4]:
climate_df.head(10)

,Date,Rain,Temp Max,Temp Min
0,01-01-1951,NaN,28.53000069,14.53999996
1,02-01-1951,NaN,28.85000038,14.47999954
2,03-01-1951,NaN,30.65999985,14.43000031
3,04-01-1951,NaN,30.13999939,14.35999966
4,05-01-1951,NaN,29.18000031,13.34000015
5,06-01-1951,NaN,28.77000046,14.18999958
6,07-01-1951,NaN,29.39999962,14.81999969
7,08-01-1951,NaN,29.59000015,14.65999985
8,09-01-1951,NaN,30.93000031,13.10000038
9,10-01-1951,NaN,30.65999985,13.55000019


In [5]:
climate_df['Date'] = pd.to_datetime(   # converts valid dates to datetime
    climate_df['Date'],
    errors='coerce',
    dayfirst=True    # handles indian DD-MM-YYYY format
)

In [6]:
mask = climate_df['Date'].isna() & climate_df['Date'].astype(str).str.isnumeric()

climate_df.loc[mask, 'Date'] = pd.to_datetime(
    climate_df.loc[mask, 'Date'].astype(int),
    unit='D',
    origin='1899-12-30'
)

# detects numeric excel dates
# converts them into real timestamps
# excel date system starts st 1899-12-30

In [7]:
climate_df[['Date', 'Rain', 'Temp Max', 'Temp Min']].sample(10)

,Date,Rain,Temp Max,Temp Min
1476,1955-01-16,NaN,30.64999962,16.28000069
8426,1974-01-26,NaN,34.13000107,15.40999985
13155,1987-01-07,NaN,28.62000084,15.23999977
15907,1994-07-21,NaN,27.56999969,23.48999977
11365,1982-02-12,NaN,31.20000076,17.82999992
12574,1985-06-05,NaN,33.24000168,25.70000076
3315,1960-01-29,NaN,30.03000069,13.06000042
25923,2021-12-22,0,32.5,18.6
24649,2018-06-27,NaN,31.13637352,25.12353134
20862,2008-02-13,NaN,31.09000015,14.90999985


In [8]:
climate_df['Date'].min(), climate_df['Date'].max()

(Timestamp('1951-01-01 00:00:00'), Timestamp('2024-06-19 00:00:00'))

In [9]:
climate_df['Rain'] = climate_df['Rain'].replace(
    ['Tr', '-----'],
    np.nan
)
# replaces non-standard missing values with NaN so pandas can process.

In [10]:
climate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26806 entries, 0 to 26805
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      26803 non-null  datetime64[ns]
 1   Rain      1177 non-null   object        
 2   Temp Max  26806 non-null  object        
 3   Temp Min  26805 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 837.8+ KB


In [11]:
climate_df['Rain'] = pd.to_numeric(climate_df['Rain'], errors='coerce')
# ensures rainfall is numeric

In [12]:
climate_df['Temp Max'] = pd.to_numeric(climate_df['Temp Max'], errors='coerce')
climate_df['Temp Min'] = pd.to_numeric(climate_df['Temp Min'], errors='coerce')
# converts temps to float, silently handling corrupted values

In [13]:
climate_df.isna().sum()  # checking missing values

Date            3
Rain        25660
Temp Max       58
Temp Min       60
dtype: int64

In [14]:
climate_df = climate_df.dropna(subset=['Date'])

In [15]:
climate_df.isna().sum()

Date            0
Rain        25660
Temp Max       57
Temp Min       59
dtype: int64

In [16]:
climate_df['Temp Max'] = climate_df['Temp Max'].interpolate()
climate_df['Temp Min'] = climate_df['Temp Min'].interpolate()

C:\Users\padil\AppData\Local\Temp\ipykernel_9848\2203703616.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_df['Temp Max'] = climate_df['Temp Max'].interpolate()
C:\Users\padil\AppData\Local\Temp\ipykernel_9848\2203703616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_df['Temp Min'] = climate_df['Temp Min'].interpolate()


In [17]:
climate_df['Rain'] = climate_df['Rain'].fillna(0)

C:\Users\padil\AppData\Local\Temp\ipykernel_9848\3075261140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_df['Rain'] = climate_df['Rain'].fillna(0)


In [ ]:
# Missing rainfall values were treated as zero, assuming non-recorded 
# rainfall indicates no precipitation, which is common in historical climate datasets.

In [18]:
climate_df.isna().sum()

Date        0
Rain        0
Temp Max    0
Temp Min    0
dtype: int64

In [19]:
climate_df['Year'] = climate_df['Date'].dt.year
climate_df['Month'] = climate_df['Date'].dt.month
# turns time into analyzable features for trends, seasonablility, dashboards

C:\Users\padil\AppData\Local\Temp\ipykernel_9848\595023152.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_df['Year'] = climate_df['Date'].dt.year
C:\Users\padil\AppData\Local\Temp\ipykernel_9848\595023152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_df['Month'] = climate_df['Date'].dt.month


In [20]:
climate_df = climate_df.sort_values('Date').reset_index(drop=True)
# ensures chronological order

In [21]:
climate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26803 entries, 0 to 26802
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      26803 non-null  datetime64[ns]
 1   Rain      26803 non-null  float64       
 2   Temp Max  26803 non-null  float64       
 3   Temp Min  26803 non-null  float64       
 4   Year      26803 non-null  int32         
 5   Month     26803 non-null  int32         
dtypes: datetime64[ns](1), float64(3), int32(2)
memory usage: 1.0 MB


## Conclusion:

- used coersion to safely parse broken data
- Distinguished true missing vs domain-missing
- applied time-aware interpolation
- made business-aligned assumptions